<a href="https://colab.research.google.com/github/sily3586/Repository_HM/blob/master/autograd_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline


Autograd: 자동 미분
===================================

Pytorch에서 모든 neural network의 중심은 "autograd"패키지라고 할 수 있다. 일단 간단한 예시로 이를 확인해보고 우리의 첫번째 neural network를 훈련시켜보자.

"autograd"패키지는 모든 텐서 연산에 대해 자동미분을 제공한다. 이것은 define-by-run 프레임워크로 돌아간다. define-by-work프레임워크란 역전파(backprop)가 코드의 실행에 따라 정의되는 것을 의미한다. 그리고 이러한 프레임워크에서 모든 개별 반복은 달라진다.

몇가지 예시와 함께 더 간단한 용어로 살펴보자.

Tensor(텐서)
--------

torch.Tensor는 패키지에서 가장 중심이되는 클래스다. torch.Tensor의 .requires_grad 속성을 True로 설정하면, 모든 연산에 대해여 추적을 시작한다. 계산이 끝나고 .backward()를 호출하면, 모든 그라디언트(gradient)를 자동으로 계산해준다. 이 텐서의 그라디언트는 .grad 속성에 누적된다.

.detach()를 호출하면, 텐서 추적을 중단하고, 이후에 일어나는 계산을 추적, 누적시키지 않는다.

기록을 추정하거나 메모리 사용을 예방하려면, 코드블럭을 with torch.no_grad(): 로 래핑. 이는 모델을 평가할 때 특히 도움이된다. 왜냐하면 모델이 학습가능한 파라미터 (requires_grad=True인)를 가지고 있을 수 있지만, 우리는 이 그라디언트가 필요하지 않기 때문이다.

*역전파가 자동으로 계산되기 때문에 학습이 끝난 뒤, 모델을 평가할 때 .detach()를 사용하거나 torch.no_grad()를 사용하여, 그라디언트 값이 변하지 않도록 해야한다는 말 같다.

자동미분을 위해 매우 중요한 클래스가 하나 더 있는데, 바로 Function 이다.

Tensor와 Function은 상호연결되어 있으며, 계산 기록 전체를 인코딩하는 비순환 그래프를 생성한다. 각각의 텐서는 텐서를 생성한 Function을 참조하는 .grad_fn 속성을 가지고 있다. (단, 사용자가 생성한 텐서는 grad_fn이 없다.)

도함수를 계산하고 싶으면 텐서의 .backward()를 호출하면 된다. 텐서가 스칼라값이면 (요소가 1개인 경우) backward()에 별다른 파라미터가 필요하지 않다. 하지만 하나 이상의 요소를 가지고 있으면, 텐서의 형태와 동일한 gradient 파라미터가 필요하다.



In [0]:
import torch

In [3]:
x = torch.ones(2, 2, requires_grad=True) #tensor를 생성하고 requires_grad 속성을 true로 설정하여 모든 연산에 대해 추적.
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [4]:
y = x + 2 #x에 대해 임의의 연산 수행 후 y에 저장.
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [5]:
print(x.grad_fn) # 이 때 x는 사용자가 직접 생성했으므로 .grad_fn 속성이 없음.
print(y.grad_fn) # 이 때 y는 연산의 결과로 생성되었기 때문에 .grad_fn 속성을 가짐.

None


In [6]:
z = y * y * 3 #y에 대해 추가적인 연산 후 z에 저장.
out = z.mean() #z의 평균값을 연산.

print(z, out) #'z'와 'out' 역시 연산의 결과로 생성되었기 때문에 .grad_fn 속성을 가짐.

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


In [7]:
#.requires_grad( ... )는 기존 텐서의 requires_grad 플래그를 제자리에서 바꿈.(덮어 쓴다는 의미인 것 같음)
#입력 플래그값이 주어지지 않으면 디폴트 값은 False. 즉 연산에 대해 추적하지 않음.
a = torch.randn(2, 2) #a 행렬을 랜덤하게 생성.
a = ((a * 3) / (a - 1)) #a에 대하여 연산한 결과를 a에 저장.
print(a.requires_grad, a.grad_fn) #a의 .requires_grad 속성을 출력. 사용자가 생성한 텐서이므로 grad_fn속성을 갖지 않음. 
a.requires_grad_(True) #a의 .requires_grad 속성의 입력 플래그를 true로 설정.
print(a.requires_grad) #a의 .requires_grad 속성을 출력.
b = (a * a).sum() #a에 대하여 연산한 결과를 b에 저장.
print(b.grad_fn) #b의 grad_fn 속성 출력.

False None
True


Gradients(그래디언트)
---------
이제 backprop(역전파)를 해보자.
"out"은 하나의 스칼라 값을 갖고있기 때문에 "out.backward(()"는 "out.backward(torch.tensor(1))"과 동일한 값을 리턴한다.

In [0]:
out.backward()

print gradients d(out)/dx




In [9]:
print(x.grad) #out을 x에 관하여 미분한 값 d(out)/dx 그래디언트를 출력.

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


"4.5"로 이뤄진 행렬이 출력.

"out" *Tensor*를 “$o$”라고 하면,

$o = \frac{1}{4}\sum_i z_i$,

$z_i = 3(x_i+2)^2$ and $z_i\bigr\rvert_{x_i=1} = 27$이다.

따라서,
$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$, 이므로


$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$이다.



autograd를 통해 더 놀라운 것들을 할 수 있다.



In [10]:
x = torch.randn(3, requires_grad=True) #(3, 3)의 사이즈를 갖는 랜덤한 텐서 생성. requires_grad 속성을 true로 설정하여 연산 추적.

y = x * 2 #x에 대하여 연산한 결과를 y에 저장.
while y.data.norm() < 1000: #y 데이터의 norm이 1000보다 작으면 y에 대한 연산의 결과를 y에 저장.
    y = y * 2

print(y)

tensor([ 901.9792, -578.3187,   33.0951], grad_fn=<MulBackward0>)


In [11]:
gradients = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float) #[0.1, 1.0, 0.0001]의 값을 가지며 데이터타입은 float인 텐서 gradients를 생성.
y.backward(gradients) #gradients에 대하여 y.backward()연산을 수행.

print(x.grad) #x의 grad를 출력

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


"requires_grad=true"를 "with torch.no_grad():"블록으로 감싸면서 tensor의 연산 기록을 추적하는 것으로부터 자동으로 미분계수를 구하는 것을 그만둘 수도 있다.


In [12]:
print(x.requires_grad) #x에 대한 requires_grad의 설정값을 출력.
print((x ** 2).requires_grad) #(x ** 2)에 대한 requires_grad의 설정값을 출력.

with torch.no_grad():
	print((x ** 2).requires_grad) #with torch.no_grad():를 통해 추적을 멈춤.

True
True
False


**Read Later:**

Documentation of ``autograd`` and ``Function`` is at
http://pytorch.org/docs/autograd

